In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
import numpy as np
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix

In [ ]:
train_data = pd.read_csv('/content/drive/Shareddrives/1:1_Rohan_Kaushikan/Dataset/train_data.csv')
val_data = pd.read_csv('/content/drive/Shareddrives/1:1_Rohan_Kaushikan/Dataset/val_data.csv')

In [ ]:
X_train = train_data.drop(['Disease_Free_Status'], axis = 1)
y_train = train_data['Disease_Free_Status']

In [ ]:
X_val = val_data.drop(['Disease_Free_Status'], axis = 1)
y_val = val_data['Disease_Free_Status']

In [ ]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
hypertrees = 400
depth = 3

In [ ]:
xgb_model_upsampled = xgb.XGBClassifier(n_estimators=hypertrees, max_depth=depth, random_state=42)
xgb_model_upsampled.fit(X_resampled, y_resampled)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [ ]:
# Predict labels on validation set
y_val_pred = xgb_model_upsampled.predict(X_val)

# Calculate accuracy and recall on validation set
accuracy_val = accuracy_score(y_val, y_val_pred)
recall_val = recall_score(y_val, y_val_pred)

print("Validation Accuracy:", accuracy_val)
print("Validation Recall:", recall_val)

Validation Accuracy: 0.8013245033112583
Validation Recall: 0.3333333333333333


In [ ]:
xgb_model = xgb.XGBClassifier(n_estimators=hypertrees, max_depth=depth, random_state=42)

In [ ]:
# Create StratifiedKFold with 5 folds
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(xgb_model, X_resampled, y_resampled, cv=kfold, scoring='accuracy')
cv_scores_recall = cross_val_score(xgb_model, X_resampled, y_resampled, cv=kfold, scoring='recall')

# Display cross-validation results
print("Cross-Validation Recall Scores:", cv_scores_recall)
print("Mean Recall:", cv_scores_recall.mean())

print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy:", cv_scores.mean())

Cross-Validation Recall Scores: [0.95588235 0.91911765 0.94890511 0.93430657 0.92647059]
Mean Recall: 0.9369364534134823
Cross-Validation Scores: [0.91208791 0.9047619  0.92307692 0.91208791 0.86397059]
Mean Accuracy: 0.9031970480499891
